<a href="https://colab.research.google.com/github/pawlishon/Python_projects/blob/master/match_excitement_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [37]:
!pip install chromedriver-autoinstaller
!pip install selenium webdriver_manager

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
MATCH_DAY = '2023-10-26'

# set up the webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
service = ChromeService(executable_path=ChromeDriverManager().install())
#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome(service=service, options = chrome_options)
url = f'https://www.nba.com/games?date={MATCH_DAY}'
driver.get(url)
driver.implicitly_wait(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [38]:
import re
matches = []
for game in soup.findAll('div', {'class': "GameCard_gc__UCI46 GameCardsMapper_gamecard__pz1rg"}):
    teams = []
    for team in game.findAll('span', {'class': 'MatchupCardTeamName_teamName__9YaBA'}):
        teams.append(team.text)
    records = []
    for record in game.findAll('p', {'class': 'MatchupCardTeamRecord_record__20YHe'}):
        records.append(record.text)
    scores = []
    for score in game.findAll('p', {'class': re.compile(r'matchupScoreCard')}):
        scores.append(score.text)
    final_result = 'NA'
    for result in game.findAll('p', {'class': 'GameCardMatchupStatusText_gcsText__PcQUX'}):
        final_result = result.text
    leaders = []
    for pra in game.findAll('td'):
        if pra.text.isdigit():
            leaders.append(int(pra.text))
    matches.append([teams[0], records[0], scores[0], teams[1], records[1], scores[1], final_result,
                    leaders[0], leaders[1], leaders[2], leaders[3], leaders[4], leaders[5]])

In [39]:
import numpy as np
matches_df = pd.DataFrame(matches, columns=['AWAY', 'AWAY_RECORD', 'AWAY_SCORE',
                                            'HOME', 'HOME_RECORD', 'HOME_SCORE', 'RESULT',
                                            'AL_PTS', 'AL_REB', 'AL_AST', 'HL_PTS',
                                            'HL_REB', 'HL_AST'])

matches_df['AWAY_RECORD'] = matches_df['AWAY_RECORD'].replace('-', '0-0')
matches_df['HOME_RECORD'] = matches_df['HOME_RECORD'].replace('-', '0-0')
matches_df['RECORD_DIFF%'] = matches_df.apply(lambda x: 0 if (int(x['AWAY_RECORD'].split('-')[0]) + int(x['AWAY_RECORD'].split('-')[1])) == 0 else abs(int(x['AWAY_RECORD'].split('-')[0]) -int(x['HOME_RECORD'].split('-')[0]))/(int(x['AWAY_RECORD'].split('-')[0]) + int(x['AWAY_RECORD'].split('-')[1])), axis=1)

# matches_df['RECORD_DIFF%'] = np.where((matches_df['AWAY_RECORD'] == '-') | (matches_df['HOME_RECORD'] == '-'), 0,
#                                       abs(int(matches_df['AWAY_RECORD'].str.split('-')[0]) -int(matches_df['HOME_RECORD'].str.split('-')[0]))/(int(matches_df['AWAY_RECORD'].str.split('-')[0]) + int(matches_df['AWAY_RECORD'].str.split('-')[1])))


# Result increase for teams with similar record
matches_df['MATCH_RESULT'] = matches_df['RECORD_DIFF%'].apply(lambda x: 1 if x < 0.25 else 0)
# result increase for close match and for big leaders scores
for i in range(len(matches_df)):
    points_diff = abs(int(matches_df.loc[i, 'AWAY_SCORE']) - int(matches_df.loc[i, 'HOME_SCORE']))
    if points_diff < 10 and points_diff > 3:
        matches_df.loc[i, 'MATCH_RESULT'] += 1
    elif points_diff <= 3:
        matches_df.loc[i, 'MATCH_RESULT'] += 2

for i in range(len(matches_df)):
    if matches_df.loc[i, 'AL_PTS'] > 50 or matches_df.loc[i, 'HL_PTS'] > 50:
        matches_df.loc[i, 'MATCH_RESULT'] += 3
    elif matches_df.loc[i, 'AL_PTS'] > 40 or matches_df.loc[i, 'HL_PTS'] > 40:
        matches_df.loc[i, 'MATCH_RESULT'] += 2
    elif matches_df.loc[i, 'AL_PTS'] > 30 or matches_df.loc[i, 'HL_PTS'] > 30:
        matches_df.loc[i, 'MATCH_RESULT'] += 1

for i in range(len(matches_df)):
    if matches_df.loc[i, 'AL_REB'] > 20 or matches_df.loc[i, 'HL_REB'] > 20:
        matches_df.loc[i, 'MATCH_RESULT'] += 1
    if matches_df.loc[i, 'AL_AST'] > 15 or matches_df.loc[i, 'HL_AST'] > 15:
        matches_df.loc[i, 'MATCH_RESULT'] += 1

# Points for OT
matches_df['MATCH_RESULT'] = matches_df.apply(lambda x: x['MATCH_RESULT'] + 1 if 'OT' in x['RESULT'] else x['MATCH_RESULT'], axis=1)

score_df = matches_df[['AWAY', 'HOME', 'MATCH_RESULT']]
print(score_df)

    AWAY    HOME  MATCH_RESULT
0  76ers   Bucks             4
1   Suns  Lakers             3
